In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf


Data Preprocessing

In [3]:
#import the dataset
Data=pd.read_csv('Churn_Modelling.csv')
x=Data.iloc[:,3:-1].values
y=Data.iloc[:,-1].values
print(x)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [4]:
#Encoding categorical data
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
x[:,2]=LE.fit_transform(x[:,-2])
print(x)

[[619 'France' 1 ... 1 1 101348.88]
 [608 'Spain' 1 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 1 ... 0 1 42085.58]
 [772 'Germany' 0 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [6]:
#Encode the geography column by hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[1])], remainder='passthrough')
x=np.array(ct.fit_transform(x))
print(x)



[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [7]:
#splitting the dataset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.2,random_state=0)

In [8]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

Building ANN

In [9]:
#initialize ann
ANN=tf.keras.models.Sequential()

#add the input layer and the first hidden layer
ANN.add(tf.keras.layers.Dense(units=6, activation='relu'))

#add the second hidden layer
ANN.add(tf.keras.layers.Dense(units=4,activation='relu'))

#add the output layer
ANN.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ANN Compiling

In [10]:
ANN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ANN Training

In [11]:
ANN.fit(x_train,y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5378 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4851 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4671 - accuracy: 0.7962
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4526 - accuracy: 0.7990
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4391 - accuracy: 0.8033
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4266 - accuracy: 0.8110
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4156 - accuracy: 0.8154
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4065 - accuracy: 0.8231
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3987 - accuracy: 0.8299
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3919 - accura

predicting the result of single obervation

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer?

In [12]:
result=ANN.predict(sc.transform([[1,0,0, 600,1,40,3,60000,2,1,1,50000]]))>0.5

1/1 [==============================] - 0s 134ms/step


In [13]:
result

array([[False]])

test set prediction

In [24]:
y_pred=ANN.predict(x_test)
y_pred=(y_pred>0.5).astype(int)
#print(y_pred)

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))


63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [25]:
#calculate the accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)      # very good score

0.861